In [1]:
import pandas as pd

Daten laden und vorbereiten

In [2]:
# Lade die CSV-Dateien mit Personen, Teilaufgaben und Projekten
personen_df = pd.read_csv("../data/personen.csv")
teilaufgaben_df = pd.read_csv("../data/teilaufgaben.csv")

In [3]:
# Wandle die Kompetenzen in eine Liste um, um Filterung zu erleichtern
personen_df["kompetenzen_liste"] = personen_df["kompetenzen"].str.split(r",\s*")

Initialisierung

In [4]:
# Belegungstracker: Speichert die bereits verplante Kapazität je Person und Monat
belegungen = {}

In [5]:
# Liste zur Speicherung der Zuweisungsergebnisse
matching_ergebnisse = []

Matching-Logik

In [6]:
def berechne_verfuegbarkeit(person, ta_start, ta_ende, belegungen):
    """
    Prüft die Verfügbarkeit einer Person für eine gegebene Zeitspanne.

    Args:
        person (pd.Series): Personendatensatz.
        ta_start (int): Startmonat (numerisch).
        ta_ende (int): Endmonat (numerisch).
        belegungen (dict): Bisherige Belegungen.

    Returns:
        tuple: (bool verfügbar, float verfügbare Gesamtkapazität, dict verfügbarkeit je Monat)
    """
    person_id = person["id"]
    verfügbar = True
    verfügbarkeit_summe = 0
    monatliche_restwerte = {}

    for monat in range(ta_start, ta_ende + 1):
        monat_label = f"{monat:02d}/2025"
        verfügbarkeit = person.get(f"verfuegbarkeit_{monat_label}", 0)
        belegt = belegungen.get(person_id, {}).get(monat_label, 0)
        rest = max(0, verfügbarkeit - belegt)
        monatliche_restwerte[monat_label] = rest

        if rest <= 0:
            verfügbar = False
            break
        verfügbarkeit_summe += rest

    return verfügbar, verfügbarkeit_summe, monatliche_restwerte


In [7]:
# Hauptloop über alle Teilaufgaben
for _, ta in teilaufgaben_df.iterrows():
    ta_start = int(ta["start"].split("/")[0])
    ta_ende = int(ta["ende"].split("/")[0])
    ta_monate = ta_ende - ta_start + 1
    ta_monatsaufwand = ta["aufwand"] / ta_monate
    restaufwand = ta["aufwand"]
    kandidaten = []

    # Filtere Personen, die die benötigte Kompetenz besitzen
    for _, person in personen_df.iterrows():
        if ta["kompetenz"] not in person["kompetenzen_liste"]:
            continue

        verfügbar, verf_summe, monatliche_restwerte = berechne_verfuegbarkeit(person, ta_start, ta_ende, belegungen)

        if verfügbar and verf_summe > 0:
            score = round(verf_summe * person["zeitbudget"], 2)
            kandidaten.append((person, score, verf_summe, monatliche_restwerte))

    # Sortiere Kandidaten nach ihrer nutzbaren Kapazität (absteigend)
    kandidaten = sorted(kandidaten, key=lambda x: x[1], reverse=True)

    # Weise Kapazität zu, bis Aufwand gedeckt ist oder keine Ressourcen mehr vorhanden
    for person, score, verf_summe, eintrag_monate in kandidaten:
        if restaufwand <= 0:
            break

        pid = person["id"]
        if pid not in belegungen:
            belegungen[pid] = {}

        verfügbarkeit_gesamt = sum(eintrag_monate.values())
        if verfügbarkeit_gesamt == 0:
            continue

        anteil = min(restaufwand, verfügbarkeit_gesamt)
        monatsanteil = anteil / ta_monate

        # Speichere Matching-Ergebnis
        matching_ergebnisse.append({
            "teilaufgabe_id": ta["teilaufgabe_id"],
            "teilaufgabe": ta["bezeichnung"],
            "projekt_id": ta["projekt_id"],
            "kompetenz": ta["kompetenz"],
            "person_id": pid,
            "name": person["name"],
            "zugewiesener_aufwand": round(anteil, 2)
        })

        # Aktualisiere belegte Verfügbarkeiten
        for monat_label in eintrag_monate:
            verbrauch = min(monatsanteil, eintrag_monate[monat_label])
            belegungen[pid][monat_label] = belegungen[pid].get(monat_label, 0) + verbrauch

        restaufwand -= anteil

    print(f"Verfügbare Kandidaten für: {ta['bezeichnung']} → {len(kandidaten)}")


Verfügbare Kandidaten für: Entwurf CAD-Baugruppe → 1
Verfügbare Kandidaten für: Simulation FEM-Struktur → 1
Verfügbare Kandidaten für: Signalverarbeitung Sensor → 0
Verfügbare Kandidaten für: Entwicklung Python-Modul → 0
Verfügbare Kandidaten für: Statistische Auswertung → 0
Verfügbare Kandidaten für: Optische Analyse → 0
Verfügbare Kandidaten für: Datenbankmodellierung → 0
Verfügbare Kandidaten für: Thermodynamische Bewertung → 0


Ergebnisse speichern

In [8]:
df_matching = pd.DataFrame(matching_ergebnisse)
df_matching.to_csv("matching_ergebnis.csv", index=False)
print("Matching abgeschlossen! Ergebnisse gespeichert in 'matching_ergebnis.csv'")

Matching abgeschlossen! Ergebnisse gespeichert in 'matching_ergebnis.csv'
